In [1]:
import pickle
import pandas as pd
import numpy as np
import os
import time
from dotenv import load_dotenv

In [2]:
from openai import OpenAI
load_dotenv()
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [3]:
user_train_path = '../../train_valid_test/user_train.pkl'
user_valid_path = '../../train_valid_test/user_valid.pkl'
user_test_path = '../../train_valid_test/user_test.pkl'

In [4]:
with open(user_train_path, 'rb') as f:
    user_train_dict = pickle.load(f)
with open(user_valid_path, 'rb') as f:
    user_valid_dict = pickle.load(f)
with open(user_test_path, 'rb') as f:
    user_test_dict = pickle.load(f)

In [5]:
user_id = 1
print(user_train_dict[user_id])
print(user_valid_dict[user_id])
print(user_test_dict[user_id])
print(len(user_train_dict[user_id][0]))

[[1961, 2028, 3105, 938, 1962, 1035, 150, 2018, 1028, 1097, 914, 2797, 1287, 2762, 1246, 661, 2918, 531, 3114, 2791, 1029, 2321, 1197, 594, 2398, 1545, 527, 1, 588, 2687, 745, 595, 2294, 783, 2355, 1907]]
[1566]
[48]
36


In [6]:
user_profile_path = '../../user_profile/cleaned_user_profile_mixtral.pkl'
with open(user_profile_path, 'rb') as f:
    user_profile_dict = pickle.load(f)
print(user_profile_dict[user_id])


            The user has a diverse taste in movies, with a preference for thought-provoking and emotionally engaging films. They enjoy dramas that explore complex themes such as mental illness, racial injustice, and the meaning of life. The user is drawn to movies that challenge their perspective and provoke deep thought, often favoring those that tackle serious and mature subjects.
            
            In addition to dramas, the user also has a soft spot for classic Disney films, as evidenced by their fondness for Cinderella. They appreciate the enchanting storytelling and timeless charm of these animated tales.
            
            The user's taste in movies also extends to comedies, as seen in their appreciation for the hilarious antics of Wallace & Gromit and the dark humor of Fargo. They are also drawn to adventure films, as demonstrated by their love for Star Wars: Episode IV - A New Hope and Back to the Future.
            
            However, the user is not a fan of 

In [7]:
movie_summary_df = pd.read_csv('../../movie_summary_25_words.csv')
movie_summary_df.head()

,movie_id,name,cleaned_genre,year,summary
0,1,Toy Story,"Animation,Children's,Comedy",1995,Toy Story is a groundbreaking animated film th...
1,2,Jumanji,"Adventure,Children's,Fantasy",1995,"""Jumanji (1995) is a thrilling adventure film ..."
2,3,Grumpier Old Men,"Comedy,Romance",1995,"""Grumpier Old Men is a hilarious and heartwarm..."
3,4,Waiting to Exhale,"Comedy,Drama",1995,"""Waiting to Exhale"" (1995) is a heartfelt dram..."
4,5,Father of the Bride Part II,Comedy,1995,Father of the Bride Part II (1995) is a heartw...


In [8]:
print(movie_summary_df[movie_summary_df['movie_id'] == 265])

     movie_id                                               name  \
262       265  Like Water for Chocolate (Como agua para choco...   

     cleaned_genre  year                                            summary  
262  Drama,Romance  1992  "Like Water for Chocolate is a visually stunni...  


In [9]:
user_df = pd.read_csv('../../users.csv')
user_df.head()

,user_id,user_profile
0,1,female k-12 student under 18 years of age
1,2,male self-employed above 56 years of age
2,3,male scientist between 25 to 34 years of age
3,4,male executive/managerial between 45 to 49 yea...
4,5,male writer between 25 to 34 years of age


In [10]:
ratings_df = pd.read_csv('../../ratings.csv')
ratings_df.head()

,user_id,movie_id,rating
0,1,3186,4
1,1,1721,4
2,1,1022,5
3,1,1270,5
4,1,2340,3


In [11]:
# reasoning_prompts = dict()
# cnt = 0
# for user, user_profile in user_profile_dict.items():
#     # if cnt < 121:
#     #     cnt += 1
#     #     continue
#     cnt += 1
#     # print(f"User: {user}")
#     prompt = """You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
#     User Profile -"""
#     ### Create the user profile
#     # final_user_profile = "The user is a "+user_df[user_df['user_id'] == user]['user_profile'].values[0]+user_profile+"\n"
#     # print(final_user_profile)
#     prompt += user_profile
#     prompt += """
#    User watched the following movies recently in the given order. List of recent movies and their description -
#     """
#     # print(prompt)
#     ### Create the like/dislike list of last 20 movies
#     # print(user_train_dict[user][0][-20:])
#     user_rating_dict = ratings_df[ratings_df['user_id'] == user]
#     # print(user_rating_dict)
#     liked_movies = []
#     disliked_movies = []
#     train_valid_movie_list = user_train_dict[user][0][-21:-1]
#     # train_valid_movie_list.append(user_valid_dict[user][0])
#     # print("Train Movie List:", len(train_valid_movie_list), train_valid_movie_list)

#     for movie in train_valid_movie_list:
#         user_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
#         # print(movie, user_movie_rating)
#         movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
#         if user_movie_rating >= 3:
#             if movie != user_valid_dict[user][0]:
#                 prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " + movie_details['summary'].values[0] + ","
#             else:
#                 ### For the last movie - ending with a . and starting new line
#                 prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"
#         else:
#             if movie != user_valid_dict[user][0]:
#                 prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + ","
#             else:
#                 ### For the last movie - ending with a . and starting new line
#                 prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"

#     ### To predict
#     prompt += """
#     Analyze all the information given in order. Do not use any information not mentioned above. Summarize the user profile and user's behavior from the given list of movies and identify the movie characteristics that he likes and dislikes in 100 words.
#     """
#     # print(user_valid_dict[user][0])
#     target_movie_details = movie_summary_df[movie_summary_df['movie_id'] == user_train_dict[user][0][-1]]
#     target_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == user_train_dict[user][0][-1]]['rating'].values[0]
#     # print("Target Movie Rating:", target_movie_details, target_movie_rating)
#     if target_movie_rating >= 3:
#         prompt += "We know that the user will like the next movie "
#         prompt += target_movie_details['name'].values[0] + "(Genre: " + target_movie_details['cleaned_genre'].values[0] + ") " +  target_movie_details['summary'].values[0]
#         prompt += f""" As an expert movie critic, analyze the user behavior, movie watching history and provide a step-by-step reasoning why the user will like {target_movie_details['name'].values[0]}. Explain your reasoning in atmost 200 words."""
#         recommendation = "\nPrediction: Yes"
#     else:
#         prompt += "We know that the user will dislike the next movie "
#         prompt += target_movie_details['name'].values[0] + "(Genre: " + target_movie_details['cleaned_genre'].values[0] + ") " +  target_movie_details['summary'].values[0]
#         prompt += f""" As an expert movie critic, analyze the user behavior, movie watching history and provide a step-by-step reasoning why the user will dislike {target_movie_details['name'].values[0]}. Explain your reasoning in atmost 200 words."""
#         recommendation = "\nPrediction: No"
#     # print('_'*100)
#     # print(f"Prompt length: {len(prompt.split())}", prompt, recommendation)
#     reasoning_prompts[user] = [prompt, recommendation]
#     # break

In [12]:
# reasoning_prompts = dict()
# cnt = 0
# for user, user_profile in user_profile_dict.items():
#     # if cnt < 121:
#     #     cnt += 1
#     #     continue
#     cnt += 1
#     # print(f"User: {user}")
#     prompt = """You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
#     User Profile -"""
#     ### Create the user profile
#     # final_user_profile = "The user is a "+user_df[user_df['user_id'] == user]['user_profile'].values[0]+user_profile+"\n"
#     # print(final_user_profile)
#     prompt += user_profile
#     prompt += """
#    User watched the following movies recently in the given order. List of recent movies and their description -
#     """
#     # print(prompt)
#     ### Create the like/dislike list of last 20 movies
#     # print(user_train_dict[user][0][-20:])
#     user_rating_dict = ratings_df[ratings_df['user_id'] == user]
#     # print(user_rating_dict)
#     liked_movies = []
#     disliked_movies = []
#     train_valid_movie_list = user_train_dict[user][0][-21:-1]
#     # train_valid_movie_list.append(user_valid_dict[user][0])
#     # print("Train Movie List:", len(train_valid_movie_list), train_valid_movie_list)

#     for movie in train_valid_movie_list:
#         user_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
#         # print(movie, user_movie_rating)
#         movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
#         if user_movie_rating >= 3:
#             if movie != user_valid_dict[user][0]:
#                 prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " + movie_details['summary'].values[0] + ","
#             else:
#                 ### For the last movie - ending with a . and starting new line
#                 prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"
#         else:
#             if movie != user_valid_dict[user][0]:
#                 prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + ","
#             else:
#                 ### For the last movie - ending with a . and starting new line
#                 prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"

#     ### To predict
#     prompt += """
#     Analyze all the information given in order. Do not use any information not mentioned above. Summarize the user's behavior from the given list of movies and identify the movie characteristics that he likes and dislikes in 50 words.
#     """
#     # print(user_valid_dict[user][0])
#     target_movie_details = movie_summary_df[movie_summary_df['movie_id'] == user_train_dict[user][0][-1]]
#     target_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == user_train_dict[user][0][-1]]['rating'].values[0]
#     # print("Target Movie Rating:", target_movie_details, target_movie_rating)
#     if target_movie_rating >= 3:
#         prompt += "We know that the user will like the next movie "
#         prompt += target_movie_details['name'].values[0] + "(Genre: " + target_movie_details['cleaned_genre'].values[0] + ") " +  target_movie_details['summary'].values[0]
#         prompt += f""" As an expert movie critic, analyze the user behavior, movie watching history and provide a step-by-step reasoning why the user will like {target_movie_details['name'].values[0]}. Explain your reasoning in atmost 200 words."""
#         recommendation = "\nPrediction: Yes"
#     else:
#         prompt += "We know that the user will dislike the next movie "
#         prompt += target_movie_details['name'].values[0] + "(Genre: " + target_movie_details['cleaned_genre'].values[0] + ") " +  target_movie_details['summary'].values[0]
#         prompt += f""" As an expert movie critic, analyze the user behavior, movie watching history and provide a step-by-step reasoning why the user will dislike {target_movie_details['name'].values[0]}. Explain your reasoning in atmost 200 words."""
#         recommendation = "\nPrediction: No"
#     # print('_'*100)
#     # print(f"Prompt length: {len(prompt.split())}", prompt, recommendation)
#     reasoning_prompts[user] = [prompt, recommendation]
#     # break

In [13]:
reasoning_prompts = dict()
cnt = 0
for user, user_profile in user_profile_dict.items():
    # if cnt < 121:
    #     cnt += 1
    #     continue
    cnt += 1
    # print(f"User: {user}")
    prompt = """You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
    User Profile -"""
    ### Create the user profile
    # final_user_profile = "The user is a "+user_df[user_df['user_id'] == user]['user_profile'].values[0]+user_profile+"\n"
    # print(final_user_profile)
    prompt += user_profile
    prompt += """
   User watched the following movies recently in the given order. List of recent movies and their description -
    """
    # print(prompt)
    ### Create the like/dislike list of last 20 movies
    # print(user_train_dict[user][0][-20:])
    user_rating_dict = ratings_df[ratings_df['user_id'] == user]
    # print(user_rating_dict)
    liked_movies = []
    disliked_movies = []
    train_valid_movie_list = user_train_dict[user][0][-19:]
    train_valid_movie_list.append(user_valid_dict[user][0])
    # print("Train Movie List:", len(train_valid_movie_list), train_valid_movie_list)

    for movie in train_valid_movie_list:
        user_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == movie]['rating'].values[0]
        # print(movie, user_movie_rating)
        movie_details = movie_summary_df[movie_summary_df['movie_id'] == movie]
        if user_movie_rating >= 3:
            if movie != user_valid_dict[user][0]:
                prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " + movie_details['summary'].values[0] + ","
            else:
                ### For the last movie - ending with a . and starting new line
                prompt += "\nLiked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"
        else:
            if movie != user_valid_dict[user][0]:
                prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + ","
            else:
                ### For the last movie - ending with a . and starting new line
                prompt += "\nDisliked " + movie_details['name'].values[0] + "(Genre: " + movie_details['cleaned_genre'].values[0] + ") " +  movie_details['summary'].values[0] + "\n"

    ### To predict
    # print(user_valid_dict[user][0])
    target_movie_details = movie_summary_df[movie_summary_df['movie_id'] == user_test_dict[user][0]]
    target_movie_rating = user_rating_dict[user_rating_dict['movie_id'] == user_test_dict[user][0]]['rating'].values[0]
    # print("Target Movie Rating:", target_movie_details, target_movie_rating)
    if target_movie_rating >= 3:
        prompt += "Now, We know that the user will like the next movie "
        prompt += target_movie_details['name'].values[0] + "(Genre: " + target_movie_details['cleaned_genre'].values[0] + ") " +  target_movie_details['summary'].values[0]
        prompt += f""" As an expert movie critic, do the following steps -
        First, Analyze the user profile and movie watching history given in order and summarize the user's behavior from the given list of movies and identify the movie characteristics that he likes and dislikes in atmost 100 words.
        Finally, Explain with reasoning why the user will like {target_movie_details['name'].values[0]} in atmost 100 words.
        Do not use any information not mentioned above."""
        recommendation = "\nPrediction: Yes"
    else:
        prompt += "Now, We know that the user will dislike the next movie "
        prompt += target_movie_details['name'].values[0] + "(Genre: " + target_movie_details['cleaned_genre'].values[0] + ") " +  target_movie_details['summary'].values[0]
        prompt += f""" As an expert movie critic, do the following steps -
        First, Analyze the user profile and movie watching history given in order and summarize the user's behavior from the given list of movies and identify the movie characteristics that he likes and dislikes in atmost 100 words.
        Finally, Explain with reasoning why the user will dislike {target_movie_details['name'].values[0]} in atmost 100 words.
        Do not use any information not mentioned above."""
        recommendation = "\nPrediction: No"
    # print('_'*100)
    # print(f"Prompt length: {len(prompt.split())}", prompt, recommendation)
    reasoning_prompts[user] = [prompt, recommendation]
    # break

In [14]:
print(len(reasoning_prompts))

6040


In [15]:
reasoning_prompts[1]

['You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.\n    User Profile -\n            The user has a diverse taste in movies, with a preference for thought-provoking and emotionally engaging films. They enjoy dramas that explore complex themes such as mental illness, racial injustice, and the meaning of life. The user is drawn to movies that challenge their perspective and provoke deep thought, often favoring those that tackle serious and mature subjects.\n            \n            In addition to dramas, the user also has a soft spot for classic Disney films, as evidenced by their fondness for Cinderella. They appreciate the enchanting storytelling and timeless charm of these animated tales.\n            \n            The user\'s taste in movies also extends to comedies, as seen in their appreciation for the hilarious antics of Wallace & Gromit and the dark humor of Fargo. They

In [16]:
reasoning_prompt_path = './reasoning_prompt_data/reasoning_prompt_test_new.pkl'
with open(reasoning_prompt_path, 'wb') as f:
    pickle.dump(reasoning_prompts, f)
    f.close()

In [17]:
content = prompt
print(content)
print('*'*100)
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "user",
      "content": content
    }
  ],
  temperature=0,
  max_tokens=256,
  top_p=0.75
)


response = response.choices[0].message.content
print(response)
len(response.split(' '))

You are an expert movie critic. You are provided with the user profile and list of recent movies that the user has watched and whether the user likes it or not.
    User Profile -
            The user has a clear preference for classic and iconic films, with a particular affinity for the Star Wars franchise, as evidenced by their inclusion of multiple Star Wars films in their liked movies list. They also enjoy adventure and sci-fi genres, with movies like E.T., Terminator 2, and Raiders of the Lost Ark among their favorites.
            
            The user's liked movies also suggest a fondness for historical dramas and war films, with Saving Private Ryan, Braveheart, and Glory all making the list. Additionally, they appreciate movies with a strong emotional core, as seen in the inclusion of Homeward Bound and She's So Lovely.
            
            On the other hand, the user's disliked movies list suggests a preference for more contemporary action films, with Metro and Die Hard b

120